In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
eff = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')

In [ ]:
eff.shape

In [ ]:
eff

In [ ]:
len(eff['patho_niv1'].unique())


In [ ]:
len(eff['cla_age_5'].unique())

In [ ]:
#On a 4 valeurs pour la variable sexe
len(eff['sexe'].unique())

In [ ]:
eff['sexe'].isin([1.0, 2.0]).sum()

In [ ]:
eff['sexe'].eq(9.0).sum()

In [ ]:
#On regroupe toutes les valeurs inconnues
#On n'a plus que 3 valeurs possibles pour sexe
eff['sexe'].replace(9.0, np.nan, inplace=True)

In [ ]:
len(eff['sexe'].unique())

In [ ]:
#Diagramme en barres sur le nombre de cas par type de maladie

# Compter le nombre de cas par type de maladie
counts = eff['patho_niv1'].value_counts()

# Créer le diagramme en barres
plt.bar(counts.index, counts.values,  width=0.6)
plt.xlabel('Type de maladie')
plt.ylabel('Nombre de cas')
plt.title('Nombre de cas par type de maladie')

# Ajouter des lignes horizontales en arrière-plan
for y in range(0, max(counts.values) + 1, 50000):  # Ajuster l'espacement des lignes selon vos besoins
    plt.axhline(y, color='gray', linestyle='--', linewidth=0.5)

# Rotation des étiquettes en diagonale
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:
#Cela pourrait être pas mal de faire un diagramme sur la proportion
# de la population de chaquee département/région qui a été touchée par une des telles maladies
#mais pour cela il faudrait la population totale de chaque département

In [ ]:
#Diagramme en barre pour le nombre de malades en fonction du sexe
#On ne voit pas de différence à cause de l'échelle
#sélectionner qq colonnes et "zoomer"
#en fait les résultats affichés me paraissent louches

# Compter le nombre de cas par type de maladie et par genre
counts = eff.groupby(['patho_niv1', 'libelle_sexe']).size().unstack(fill_value=0)

# Créer le diagramme en barres
ax = counts.plot(kind='bar', stacked=False)

# Ajouter des lignes horizontales dans l'arrière-plan
for y in range(50, max(counts.max())+1, 50000):
    ax.axhline(y=y, color='gray', linestyle='--', linewidth=0.5)

# Orienter le texte sur l'axe des x 
plt.xticks(rotation=45, ha='right')


plt.xlabel('Type de maladie')
plt.ylabel('Nombre de cas')
plt.title('Nombre de cas par type de maladie chez les individus par genre')
plt.legend(title='Genre', loc='upper right')

plt.show()


In [ ]:
# Maladies les plus répandues chez les hommes

patho_niv3_counts = eff[eff['sexe'] == 1.0]['patho_niv2'].value_counts()

# Sélectionner les 10 maladies les plus répandues
top_10_maladies = patho_niv3_counts.head(10)

# Calculer le pourcentage d'hommes ayant contracté chaque maladie parmi l'ensemble de la population soignée
pourcentages = (top_10_maladies / len(eff[eff['sexe'] == 1.0])) * 100

# Créer le graphique en barres
pourcentages.plot(kind='bar', color='mediumaquamarine')


plt.xlabel('Type de maladie (patho_niv2)')
plt.ylabel('Parmi ceux qui ont reçu des soins,\n pourcentage d\'hommes concerné par la maladie')
plt.title('Les 10 maladies les plus répandues chez les hommes')
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:
# il y a autant de personnes pour chaque classe d'âge comme le montre le graph
# faire attention à l'interprétation causale !!!!

# Compter le nombre d'individus par tranche d'âge
age_counts = eff['cla_age_5'].value_counts()

# Calculer la proportion d'individus dans chaque tranche d'âge
age_proportions = age_counts / age_counts.sum()

# Trier les proportions par index (tranches d'âge) en ordre croissant
age_proportions = age_proportions.sort_index()

# Créer le diagramme en barres
ax = age_proportions.plot(kind='bar', color='green')

# Ajouter des étiquettes aux barres
for i, proportion in enumerate(age_proportions):
    ax.text(i, proportion + 0.01, f'{proportion:.2%}', ha='center', va='bottom')

plt.xlabel('Tranche d\'âge')
plt.ylabel('Proportion d\'individus')
plt.title('Proportion d\'individus dans chaque tranche d\'âge')
plt.show()


In [ ]:
#valide le fait qu'il y a autant d'indiv dans chaque classe d'âge
for x in eff['cla_age_5'].unique():
   print(x, eff['cla_age_5'].eq(x).sum())

In [ ]:
data_pollution = pd.read_csv("base_pollution.csv", sep = ",", encoding = 'utf-8', engine = 'python')

In [ ]:
data_pollution

In [ ]:
pollution_group_dep = pd.read_csv("pollution_group_dep.csv", sep = ",", encoding = 'utf-8', engine = 'python')

In [ ]:
pollution_group_reg = pd.read_csv("pollution_group_reg.csv", sep = ",", encoding = 'utf-8', engine = 'python')

In [ ]:
pollution_group_dep

In [ ]:
pollution_group_reg

In [ ]:
# Sommez les colonnes 3 à 12 pour chaque région
pollution_agregee = pollution_group_reg.groupby('Nom de la région').sum().iloc[:, 1:11]

# Créez un histogramme
pollution_agregee.plot(kind='bar', stacked=True)
plt.xlabel('Nom de la région')
plt.ylabel('Niveau agrégé de pollution')
plt.title('Niveau agrégé de pollution par région')
plt.legend(title='Variables de pollution', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:
x = input("Entrer un numéro de département (mettre un 0 devant si c'est un chiffre) : ")

In [ ]:
# Sélectionnez les données pour le département x
pollution_dep_91 = pollution_group_dep[pollution_group_dep['DEP'] == x]

# Sélectionnez les colonnes correspondant aux sources de pollution
sources_pollution = pollution_dep_91.columns[1:10]

# Créez un diagramme en barres
plt.bar(sources_pollution, pollution_dep_91.iloc[0, 1:10])
plt.xlabel('Sources de pollution')
plt.ylabel('Niveau de pollution')
plt.title('Niveau de pollution par source pour le département {}'.format(x))
plt.xticks(rotation=45, ha='right')

plt.show()


In [ ]:
# Fusion des tables effectifs et pollution_group_dep
malades_et_pollution_dep = pd.merge(eff, pollution_group_dep, 
                    left_on=[eff['dept']], 
                    right_on=[pollution_group_dep['DEP']], 
                    how='inner').drop(columns=['key_0'])

In [ ]:
malades_et_pollution_dep

In [ ]:
# Fusion des tables effectifs et pollution_group_reg
#En cas de stat sur les corrélation entre pollution et maladie au niveau de la région
malades_et_pollution_reg = pd.merge(eff, pollution_group_reg, 
                    left_on=[eff['region']], 
                    right_on=[pollution_group_reg['REG']], 
                    how='inner').drop(columns=['key_0'])

In [ ]:
malades_et_pollution_reg